In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import glob
import pandas as pd
import PySimpleGUI as sg
from collections import defaultdict
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import numpy as np
from gensim import models

!apt-get install -y xvfb # Install X Virtual Frame Buffer
import os
os.system('Xvfb :1 -screen 0 1600x1200x16  &')    # create virtual display with size 1600x1200 and 16 bit color. Color can be changed to 24 or 8
os.environ['DISPLAY']=':1.0'    # tell X clients to use our virtual DISPLAY :1.0

In [4]:
import pandas as pd

In [6]:
dataf = pd.read_csv('C:/Users/LeonFremz/Desktop/profileTables.csv')

print(dataf.shape)

(19475, 4)


In [7]:
datafText= dataf['Title']
datafText

0        Adaptive Filtering-Based Pseudo Open-Loop Thre...
1        Coordinate Transformation-Free Observer-based ...
2        Demodulation Type Single-Phase PLL with DC Off...
3        Frequency Adaptive Parameter Estimation of Unb...
4        Gain Normalized Adaptive Observer For Three-Ph...
                               ...                        
19470    A Sensitivity Approach for Eliminating Clashes...
19471    Using parametric sensitivity analysis to detec...
19472               Design Intent for CAD model assemblies
19473    Implementing a maths support system for first-...
19474    Sensitivity approach for automatically elimina...
Name: Title, Length: 19475, dtype: object

In [14]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
#Eliminating stopwords 
#this will contain a list of all the words in the corpus

datafText_words = []
# Tokenize a paragraph into sentences and each sentence in to
# words
for c in datafText:
    for sent in sent_tokenize(c):
      word_tokens = word_tokenize(sent)
      datafText_words += word_tokens

len(datafText)

In [16]:
#inspecting the words in the datafText a bit
datafText_words.index('the'), datafText_words.index('The')

ValueError: 'the' is not in list

In [17]:
datafText_words.index('so'), datafText_words.index('So')

ValueError: 'so' is not in list

In [18]:
lower_case_words = set([ x.lower() for x in datafText_words])
len(lower_case_words)

0

In [20]:
from nltk.corpus import stopwords
#remove stopwords
stwords = set(stopwords.words('english'))
#using set difference to calculate stopwords from our words

stopfree_words = lower_case_words - stwords
len(stopfree_words)




AttributeError: module 'nltk' has no attribute 'data'

In [ ]:
#Stemming
from nltk.stem import snowball

stemmer = snowball.SnowballStemmer('english')
stemmed_words = set([stemmer.stem(x) for x in stopfree_words])
len(stemmed_words)


18003

In [ ]:
# Lets look at some of our words
list(stemmed_words)[-10:]

['gasoline-blend',
 'perfect',
 'blue',
 'phase',
 'silicofcm',
 'mycobacteri',
 'disenchant',
 'terahertz',
 'schumpet',
 'culturalist']

In [ ]:
#constructing the index
# Our index is a map of word -> documents it is found in
inverted_index = defaultdict(set)

# We maintain the reference to the document by its index in the corpus list
for docid, c in enumerate(datafText):
    for sent in sent_tokenize(c):
        for word in word_tokenize(sent):
            word_lower = word.lower()
            if word_lower not in stwords:
                word_stem = stemmer.stem(word_lower)
                # We add the document to the set againt the word in our
                # index
                inverted_index[word_stem].add(docid)

len(inverted_index.keys())

18003

In [21]:
#searching using the index

def process_and_search(query):
    matched_documents = set()
    for word in word_tokenize(query):
        word_lower = word.lower()
        if word_lower not in stwords:
            word_stem = stemmer.stem(word_lower)
            matches = inverted_index.get(word_stem)
            if matches:
                # The operator |= is a short hand for set union
                matched_documents |= matches
    return matched_documents


In [ ]:
process_and_search("phase")

#the ouput is where the word "phase" appears in documents

{13,
 42,
 1101,
 1113,
 1119,
 1122,
 1925,
 1967,
 1969,
 2271,
 2273,
 2636,
 3139,
 3141,
 3149,
 3150,
 3156,
 3160,
 3166,
 3167,
 3169,
 3173,
 3174,
 3181,
 3196,
 3207,
 3213,
 3226,
 3227,
 3335,
 3337,
 3386,
 3707,
 3970,
 4138,
 4159,
 4185,
 4394,
 5136,
 5142,
 5238,
 5465,
 5552,
 5609,
 5634,
 5635,
 5658,
 5664,
 5775,
 5785,
 5798,
 5805,
 5808,
 5812,
 5975,
 5976,
 5981,
 5987,
 6012,
 6400,
 6423,
 6462,
 6594,
 6596,
 6951,
 7149,
 7495,
 7776,
 7943,
 7950,
 8198,
 8220,
 8223,
 8978,
 8983,
 8984,
 8987,
 8989,
 8994,
 8998,
 8999,
 9041,
 9050,
 9070,
 9077,
 9314,
 9364,
 9562,
 9626,
 9648,
 9978,
 10083,
 10805,
 10948,
 11364,
 11557,
 11933,
 12153,
 12165,
 12351,
 12489,
 13222,
 13229,
 13231,
 13233,
 13414,
 14058,
 14132,
 14253,
 15242,
 15243,
 15864,
 16029,
 16031,
 16059,
 16546,
 17014,
 17019,
 17031,
 17072,
 17295,
 17780,
 17928,
 17929,
 17930,
 17934,
 18079,
 18475,
 18477,
 18500,
 18503,
 18504}

In [ ]:
process_and_search("blue")

{6582, 9716, 12731, 14140}

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
import string

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
entry = dataf.loc[0,:].copy()

In [ ]:
entry

Name                                                      Hafiz Ahmed
profile_URL         https://pureportal.coventry.ac.uk/en/persons/a...
publications_url    https://pureportal.coventry.ac.uk/en/persons/a...
Title               Adaptive Filtering-Based Pseudo Open-Loop Thre...
Name: 0, dtype: object

In [ ]:
def process_string(content):
  content = content.lower() #to lowercase
  content = content.translate(str.maketrans('', '', string.punctuation)) #strip punctuation
  return content

In [ ]:

process_string(entry.Title)

'adaptive filteringbased pseudo openloop threephase gridsynchronization technique '

In [ ]:

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
print("Apple: {}\n Run: {}\n Happy: {}" .format(get_wordnet_pos("apple"), get_wordnet_pos("run"), get_wordnet_pos("happy")))

Apple: n
 Run: v
 Happy: a


In [ ]:
stop = stopwords.words('english')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
#################################################################################################

In [ ]:
lem = WordNetLemmatizer()

def stop_lemmatize(doc):
    tokens = nltk.word_tokenize(doc)
    tmp = ""
    for w in tokens:
        if w not in stop:
            tmp += lem.lemmatize(w, get_wordnet_pos(w)) + " "
    return tmp

In [ ]:
stop_lemmatize(doc = entry.Title)

'Adaptive Filtering-Based Pseudo Open-Loop Three-Phase Grid-Synchronization Technique '

In [ ]:
def process_string(content):
  content = content.lower() #to lowercase
  content = content.translate(str.maketrans('', '', string.punctuation)) #strip punctuation
  content = stop_lemmatize(content)
  return content

In [ ]:
%time process_string(entry.Title)

CPU times: user 4.72 ms, sys: 0 ns, total: 4.72 ms
Wall time: 4.83 ms


'adaptive filteringbased pseudo openloop threephase gridsynchronization technique '

In [ ]:
meta = dataf.copy()

In [ ]:
def transform_dataf(dataf):
  dataf['Title'] = dataf['Title'].apply(process_string)
  dataf['Name'] = dataf['Name'].apply(process_string)

In [ ]:
%time transform_dataf(meta)

CPU times: user 39.3 s, sys: 1.67 s, total: 40.9 s
Wall time: 40.8 s


In [ ]:
meta.head(5)

,Name,profile_URL,publications_url,Title,text
0,hafiz ahmed,https://pureportal.coventry.ac.uk/en/persons/a...,https://pureportal.coventry.ac.uk/en/persons/a...,adaptive filteringbased pseudo openloop threep...,adaptive filteringbased pseudo openloop threep...
1,hafiz ahmed,https://pureportal.coventry.ac.uk/en/persons/a...,https://pureportal.coventry.ac.uk/en/persons/a...,coordinate transformationfree observerbased ad...,coordinate transformationfree observerbased ad...
2,hafiz ahmed,https://pureportal.coventry.ac.uk/en/persons/a...,https://pureportal.coventry.ac.uk/en/persons/a...,demodulation type singlephase pll dc offset re...,demodulation type singlephase pll dc offset re...
3,hafiz ahmed,https://pureportal.coventry.ac.uk/en/persons/a...,https://pureportal.coventry.ac.uk/en/persons/a...,frequency adaptive parameter estimation unbala...,frequency adaptive parameter estimation unbala...
4,hafiz ahmed,https://pureportal.coventry.ac.uk/en/persons/a...,https://pureportal.coventry.ac.uk/en/persons/a...,gain normalize adaptive observer threephase sy...,gain normalize adaptive observer threephase sy...


In [ ]:

meta['Text'] = meta['Title'] + " " + meta['Name']
drop_cols = ['profile_URL', 'publications_url']
meta = meta.drop(drop_cols, axis=1)

KeyError: ignored

In [ ]:
meta.head(5)

,Name,Title,text,Text
0,hafiz ahmed,adaptive filteringbased pseudo openloop threep...,adaptive filteringbased pseudo openloop threep...,adaptive filteringbased pseudo openloop threep...
1,hafiz ahmed,coordinate transformationfree observerbased ad...,coordinate transformationfree observerbased ad...,coordinate transformationfree observerbased ad...
2,hafiz ahmed,demodulation type singlephase pll dc offset re...,demodulation type singlephase pll dc offset re...,demodulation type singlephase pll dc offset re...
3,hafiz ahmed,frequency adaptive parameter estimation unbala...,frequency adaptive parameter estimation unbala...,frequency adaptive parameter estimation unbala...
4,hafiz ahmed,gain normalize adaptive observer threephase sy...,gain normalize adaptive observer threephase sy...,gain normalize adaptive observer threephase sy...


In [ ]:

def transform_dataf(dataf):
  dataf = dataf
  dataf['Title'] = dataf['Title'].apply(process_string)
  dataf['Name'] = dataf['Name'].apply(process_string)
  dataf['text'] = dataf['Title'] + " " + dataf['Name']
  drop_cols = ['profile_URL', 'publications_url']
  dataf = dataf.drop(drop_cols, axis=1)
  return dataf

In [ ]:
#build index

In [ ]:
entry = meta.loc[0,:].copy()
print(entry)
index_test = {}

Name                                          hafiz ahmed 
Title    adaptive filteringbased pseudo openloop threep...
text     adaptive filteringbased pseudo openloop threep...
Text     adaptive filteringbased pseudo openloop threep...
Name: 0, dtype: object


In [ ]:
words = entry.Text.split()
Text = entry.Text

In [ ]:
word = words[0]
sample = {word: [Text]}
print(sample)

{'adaptive': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique  hafiz ahmed ']}


In [ ]:
for word in words:
  if word in index_test.keys():
    if Title not in index_test[word]:
      index_test[word].append(Title)
  else:
    index_test[word] = [Title]

In [ ]:
print(index_test)

{'adaptive': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'filteringbased': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'pseudo': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'openloop': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'threephase': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'gridsynchronization': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'technique': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'hafiz': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'ahmed': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique ']}


In [ ]:

def index_it(entry, index):
  words = entry.Title.split()
  Title = entry.Title
  for word in words:
    if word in index.keys():
      if Title not in index[word]:
        index[word].append(Title)
    else:
      index[word] = [Title]
  return index

In [ ]:
ind = index_it(entry=entry, index= {})
print(ind)

{'adaptive': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'filteringbased': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'pseudo': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'openloop': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'threephase': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'gridsynchronization': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique '], 'technique': ['adaptive filteringbased pseudo openloop threephase gridsynchronization technique ']}


In [ ]:
def index_all(df, index):
  for i in range(len(df)):
    entry = df.loc[i,:]
    index = index_it(entry = entry, index = index)
  return index

In [ ]:
index = index_all(meta_processed, index = {})
len(index)

19835

In [ ]:
def build_index(dataf, index):
    to_add = transform_df(dataf)
    index = index_all(df = to_add, index = index)
    return index

In [ ]:

idx = build_index(dataf = dataf, index = {})

In [ ]:
len(idx)

19835

In [ ]:
import json

with open("/content/drive/MyDrive/information retrieval/7071CEM Information Retrieval/CW/New folder/pureportal_survey_json.json", 'w') as fp:
    json.dump(idx, fp, sort_keys=True, indent=4)

In [ ]:
with open('/content/drive/MyDrive/information retrieval/7071CEM Information Retrieval/CW/New folder/pureportal_survey_json.json', 'r') as f:
    pureportal_survey_json = json.load(f)

In [ ]:
word2vec = pureportal_survey_json

In [ ]:
print(words)

['adaptive', 'filteringbased', 'pseudo', 'openloop', 'threephase', 'gridsynchronization', 'technique', 'hafiz', 'ahmed']


In [ ]:

# Python program to generate word vectors using Word2Vec
  
# importing all necessary modules
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
import warnings

In [ ]:
#  Reads ‘alice.txt’ file
sample = open("/content/drive/MyDrive/information retrieval/1pureportal_survey_json.txt", "r")
s = sample.read()

In [ ]:
# Replaces escape character with space
f = s.replace("\n", " ")
  
data = []

In [ ]:
# iterate through each sentence in the file
for i in sent_tokenize(f):
    temp = []
      
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())
  
    data.append(temp)

In [ ]:
# Create CBOW model
model1 = gensim.models.Word2Vec(data, min_count = 1, 
                              size = 100, window = 5)

In [ ]:
# Print results
print(sample, model1.similarity('vasile', 'trump'))

<_io.TextIOWrapper name='/content/drive/MyDrive/information retrieval/1pureportal_survey_json.txt' mode='r' encoding='UTF-8'> 0.19963203


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


In [ ]:
#################################################################################

In [ ]:
test = "filtering based"

In [ ]:
print("User query: {}." .format(test))
test_norm = process_string(test)
print("Normalized query: {}." .format(test_norm))

User query: filtering based.
Normalized query: filter base .


In [ ]:
test_split = test_norm.split()

In [ ]:
def process_query(query):
  norm = process_string(query)
  return norm.split()

In [ ]:
retrieved = []
for word in test_split:
  if word in index.keys():
    retrieved.append(index[word])

In [ ]:
def lists_intersection(lists):
  intersect = list(set.intersection(*map(set, lists)))
  intersect.sort()
  return intersect

In [ ]:
result = lists_intersection(retrieved)
print(result)

['adaptive filteringbased pseudo openloop threephase gridsynchronization technique ']


In [ ]:
def search_googleish(query, index=idx):
  query_split = process_query(query)
  retrieved = []
  for word in query_split:
    if word in index.keys():
      retrieved.append(index[word])
  if len(retrieved)>0:
    result = lists_intersection(retrieved)
  else:
      result = [0]
  return result

In [ ]:
result_IDs = search_googleish("Trump", index)
print(result_IDs)

['native american racism age donald trump historical contemporary perspective ', 'president trump christian right ']


In [ ]:
#Retrieve meta-data

In [ ]:
#in real setting we'll read the database from file here
#meta = pd.read_csv("database.csv")

#this is our database
meta = dataf.drop(['text'], axis=1).copy()
meta.head(5)

,Name,profile_URL,publications_url,Title
0,hafiz ahmed,https://pureportal.coventry.ac.uk/en/persons/a...,https://pureportal.coventry.ac.uk/en/persons/a...,adaptive filteringbased pseudo openloop threep...
1,hafiz ahmed,https://pureportal.coventry.ac.uk/en/persons/a...,https://pureportal.coventry.ac.uk/en/persons/a...,coordinate transformationfree observerbased ad...
2,hafiz ahmed,https://pureportal.coventry.ac.uk/en/persons/a...,https://pureportal.coventry.ac.uk/en/persons/a...,demodulation type singlephase pll dc offset re...
3,hafiz ahmed,https://pureportal.coventry.ac.uk/en/persons/a...,https://pureportal.coventry.ac.uk/en/persons/a...,frequency adaptive parameter estimation unbala...
4,hafiz ahmed,https://pureportal.coventry.ac.uk/en/persons/a...,https://pureportal.coventry.ac.uk/en/persons/a...,gain normalize adaptive observer threephase sy...


In [ ]:

def connect_id_df(retrieved_id, dataf):
    return dataf[dataf.Title.isin(retrieved_id)].reset_index(drop=True)

In [ ]:
result_meta = connect_id_df(result_IDs, meta)
result_meta.head(5)

,Name,profile_URL,publications_url,Title
0,darren reid,https://pureportal.coventry.ac.uk/en/persons/d...,https://pureportal.coventry.ac.uk/en/persons/d...,native american racism age donald trump histor...
1,chris shannahan,https://pureportal.coventry.ac.uk/en/persons/c...,https://pureportal.coventry.ac.uk/en/persons/c...,president trump christian right


In [ ]:

def average_vectors(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.vocab]
    if len(doc) == 0:
      return np.zeros(300)
    else:
      return np.mean(word2vec_model[doc], axis=0)

In [ ]:
def cos_similarity(a, b):
  dot = np.dot(a, b)
  norma = np.linalg.norm(a)
  normb = np.linalg.norm(b)
  cos = dot / (norma * normb)
  return(cos)

In [ ]:

query_vec = average_vectors(word2vec, test_split)

AttributeError: ignored

In [ ]:
result_vecs = connect_id_df(result_IDs, dataf)

In [ ]:
cos_sim = []
for i in range(len(result_vecs)):
  doc_vec = dataf.loc[i,:].drop(['profile_URL'])
  cos_sim.append(cos_similarity(doc_vec, query_vec))
result_meta['rank'] = cos_sim

In [ ]:
def rank_results(query, results):
  query_norm = process_query(query)
  #query_vec = average_vectors(word2vec, query_norm)
  result_vecs = connect_id_df(results.Title, meta)
  cos_sim = []
  for i in range(len(result_vecs)):
    #doc_vec = result_vecs.loc[i,:].drop(['ID'])
    cos_sim.append(cos_similarity(doc_vec, query_vec))
    results['rank'] = cos_sim
    results = results.sort_values('rank', axis=0)
  return results

In [ ]:

final_result = rank_results("Trump", result_meta)

NameError: ignored

In [ ]:
def print_results(result_dataf):
  for i in range(len(result_dataf)):
    res = dataf.loc[i,:]
    print(res.Title)
    print(res.Name)
    if i == len(result_dataf):
        print(res.publications_url)
    else:
        print("{}\n" .format(res.publications_url))

In [ ]:
print_results(final_result)

NameError: ignored

In [ ]:
def search(query, dat=None):
  result = search_googleish(query)
  result = connect_id_df(result, meta)

  if dat is not None:
    result = filter_date(dat, result)

  print_results(result)

In [ ]:
query = input("Search for:")
search(query)

Search for:trump


NameError: ignored

In [ ]:
import PySimpleGUI as sg
sg.theme_input_text_color('#000000')
import os
os.system('Xvfb :1 -screen 0 1600x1200x16  &')    # create virtual display with size 1600x1200 and 16 bit color. Color can be changed to 24 or 8
os.environ['DISPLAY']=':1.0'    # tell X clients to use our virtual DISPLAY :1.0


layout = [
    [sg.Text("Hello")],
    [sg.Button("ok")]
]

window = sg.Window("demo", layout)

while True:
  event, values = window.read()

  if event == "ok" or event == sg.WIN_CLOSED:
    break
window.close()


In [ ]:
# Interface Code #
##
val_list = []
sg.theme_input_text_color('#000000')

layout = [[sg.Text("Type a query")],
          [sg.Input(key='query')],
          [sg.Button('Search')],
          [sg.Text(size=(20,1))],
          [sg.Table(values=val_list, headings=["Name", "Title", "publications_url"], background_color='gray',
                    auto_size_columns=False, display_row_numbers=False,justification='left', num_rows=15, col_widths = [5,20,40,100,10,20],
                    key='TABLE', row_height=25, enable_events=False)],
        [sg.Button('Exit')]]

window = sg.Window('Search Engine', layout)

while True:
    event, values = window.read()

    if event == sg.WINDOW_CLOSED or event == 'Exit':
        break
    
    query = values['query']

    try:
        idx, results = search(index, data, 30, query, "Name", "Title")
        print(results)

        window['TABLE'].update(values = results)
    except:
        print("error")

window.close()

In [ ]:
!pip install PySimpleGUI